# Algoritma 15.1 (Simple Filter Method)

In [2]:
import numpy as np

def objective_function(x):
    """Example objective function f(x)"""
    return np.sum(x**2)  # Example: Quadratic function

def constraint_violation(x):
    """Example constraint violation function h(x)"""
    return max(0, np.sum(x) - 1)  # Example: Sum of x should not exceed 1

def step_generation(x, delta):
    """Generate a trial step (This can be a trust-region step, line search step, etc.)"""
    return -delta * np.ones_like(x)  # Example: Simple descent direction

def feasibility_restoration(x):
    """Feasibility restoration phase (if step-generation fails)"""
    return np.zeros_like(x)  # Example: Projecting to feasible space (dummy)

def filter_method(x0, delta0, max_iter=100, tol=1e-6):
    """General Filter Method Implementation"""
    x = x0
    delta = delta0
    k = 0
    filter_set = set()  # Stores (f_k, h_k) pairs

    while k < max_iter:
        # Check if the step-generation subproblem is feasible
        pk = step_generation(x, delta)
        if pk is None:
            x_new = feasibility_restoration(x)
        else:
            x_trial = x + pk
            f_trial = objective_function(x_trial)
            h_trial = constraint_violation(x_trial)

            # Check if (f+, h+) is acceptable to the filter
            is_acceptable = all(f_trial < f and h_trial < h for f, h in filter_set)
            
            if is_acceptable:
                x = x_trial
                filter_set.add((f_trial, h_trial))
                delta = max(delta, 1.2 * delta)  # Increase trust-region radius
                
                # Remove dominated pairs from the filter
                filter_set = {(f, h) for f, h in filter_set if not (f >= f_trial and h >= h_trial)}
            else:
                delta *= 0.5  # Reduce trust-region radius if step is rejected
        
        # Convergence check
        if np.linalg.norm(pk) < tol:
            break
        
        k += 1
    
    return x

# Example usage
x0 = np.array([2.0, 2.0])  # Initial point
delta0 = 1.0  # Initial trust-region radius
solution = filter_method(x0, delta0)
print("Optimal solution:", solution)


Optimal solution: [-4.14089866e+08 -4.14089866e+08]


# Algoritma 15.2 (Generic Algorithm with Second-Order Correction)

In [4]:
def phi(x, mu):
    # Define your objective function here
    return np.linalg.norm(x)**2  # Example: quadratic function

def D_phi(x, p):
    # Define the directional derivative of phi at x along p
    return 2 * np.dot(x, p)  # Example: gradient of quadratic function

def compute_search_direction(x):
    # Define how to compute search direction p_k
    return -2 * x  # Example: steepest descent direction

def compute_second_order_correction(x, p):
    # Define the second-order correction direction \hat{p}_k
    return -0.5 * p  # Example: some small correction

def algorithm_15_2(x0, eta=0.1, tau1=0.1, tau2=0.9, max_iters=100, tol=1e-6):
    x = np.array(x0, dtype=float)
    k = 0
    
    while k < max_iters:
        p = compute_search_direction(x)
        alpha_k = 1.0
        newpoint = False
        
        while not newpoint:
            if phi(x + alpha_k * p, 0) <= phi(x, 0) + eta * alpha_k * D_phi(x, p):
                x_new = x + alpha_k * p
                newpoint = True
            elif alpha_k == 1:
                p_hat = compute_second_order_correction(x, p)
                if phi(x + p + p_hat, 0) <= phi(x, 0) + eta * D_phi(x, p):
                    x_new = x + p + p_hat
                    newpoint = True
                else:
                    alpha_k *= np.random.uniform(tau1, tau2)  # Choose new alpha_k
            else:
                alpha_k *= np.random.uniform(tau1, tau2)  # Choose new alpha_k
        
        if np.linalg.norm(x_new - x) < tol:
            break  # Convergence criterion
        
        x = x_new
        k += 1
    
    return x

# Example usage:
x0 = [1.0, 2.0]
result = algorithm_15_2(x0)
print("Optimal solution:", result)


Optimal solution: [0. 0.]


# Algorithm 15.3 (Watchdog)

In [6]:
def watchdog_algorithm(phi, grad_phi, direction_func, x0, eta=0.3, tol=1e-6, max_iter=100):
    k = 0
    S = set()
    x_k = x0
    
    while k < max_iter:
        p_k = direction_func(x_k)  # Compute search direction
        x_k1 = x_k + p_k  # Step update
        
        if phi(x_k1) <= phi(x_k) + eta * np.dot(grad_phi(x_k), p_k):
            k += 1
            S.add(k)
            x_k = x_k1
        else:
            p_k1 = direction_func(x_k1)
            alpha_k1 = line_search(phi, grad_phi, x_k1, p_k1, eta)
            x_k2 = x_k1 + alpha_k1 * p_k1
            
            if phi(x_k1) <= phi(x_k) or phi(x_k2) <= phi(x_k) + eta * np.dot(grad_phi(x_k), p_k):
                k += 2
                S.add(k)
                x_k = x_k2
            elif phi(x_k2) > phi(x_k):
                alpha_k = line_search(phi, grad_phi, x_k, p_k, eta)
                x_k3 = x_k + alpha_k * p_k
                k += 3
                S.add(k)
                x_k = x_k3
            else:
                p_k2 = direction_func(x_k2)
                alpha_k2 = line_search(phi, grad_phi, x_k2, p_k2, eta)
                x_k3 = x_k2 + alpha_k2 * p_k2
                k += 3
                S.add(k)
                x_k = x_k3
        
        if np.linalg.norm(grad_phi(x_k)) < tol:
            break
    
    return x_k, S

def line_search(phi, grad_phi, x, p, eta):
    """A simple backtracking line search satisfying Armijo condition."""
    alpha = 1.0
    beta = 0.5  # Reduction factor
    while phi(x + alpha * p) > phi(x) + eta * alpha * np.dot(grad_phi(x), p):
        alpha *= beta
    return alpha

# Example usage with a quadratic function
def phi(x):
    return np.dot(x, x)

def grad_phi(x):
    return 2 * x

def direction_func(x):
    return -grad_phi(x)

x0 = np.array([1.0, 1.0])
x_opt, S = watchdog_algorithm(phi, grad_phi, direction_func, x0)
print("Optimal x:", x_opt)
print("S:", S)


Optimal x: [0. 0.]
S: {2}
